# Finetune model using AWS Knowledgebase

In [1]:
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm synthetic-data-kit==0.0.3
else:
    try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
    except: get_numpy = "numpy"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.10.1", "triton==3.2.0") if is_t4 else ("vllm", "triton")
    !uv pip install -qqq --upgrade         unsloth {get_vllm} {get_numpy} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
    !uv pip install synthetic-data-kit==0.0.3
!uv pip install transformers==4.55.4

In [2]:
from datasets import Dataset
import pandas as pd

conversations = pd.read_json("dataset.json").reset_index(drop = True)
dataset = Dataset.from_pandas(conversations)

In [3]:
dataset[0]

{'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'What are the most significant changes in ransomware tactics reported for 2024, and how can an AWSbased organization detect and mitigate these new attack vectors?',
   'role': 'user'},
  {'content': 'The OpenText 2024 report highlights a shift from traditional encryptiononly ransomware to **dataexfiltrationonly** (extortionware) and **double/triple extortion** strategies. Attackers now prefer to steal sensitive data and threaten disclosure rather than encrypt it, which bypasses the effectiveness of robust backup programs. They also combine encryption, exfiltration, and direct pressure on customers or regulators, increasing reputational damage. In AWS environments, these tactics can be detected by monitoring **CloudTrail** for anomalous API calls that download large volumes of S3 objects or use `GetObject` and `CopyObject` excessively. GuardDuty can flag unusual data transfer patterns, and Amazon 

In [4]:
dataset[1]

{'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'How should an organization address the risk of supplychain ransomware attacks identified in the report, and what AWS services can support a secure vendormanagement program?',
   'role': 'user'},
  {'content': 'Supplychain ransomware originates from compromised thirdparty software or infrastructure. The report notes that 62f% of recent attacks were traced to a software supplychain partner, prompting many firms to audit vendors annually. In AWS, a secure vendormanagement program begins with **AWS Artifact** to obtain SOCf2, ISOf27001, and other compliance reports from service providers. Use **AWS Marketplace**s verified products and integrate with **AWS CodeBuild** or **CodePipeline** to automatically scan thirdparty code with **Amazon CodeGuru Reviewer** and **AWS Security Hub** findings. Implement **AWS Secrets Manager** to rotate and manage credentials for vendor APIs, and enforce **leastprivil

### Fine-tuning Synthetic Dataset with Unsloth

In [5]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3-270m-it",
    max_seq_length = 4096, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = True, # [NEW!] We have full finetuning now!
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 01-04 12:40:41 [__init__.py:216] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.7: Fast Gemma3 patching. Transformers: 4.55.4. vLLM: 0.10.2.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 23.559 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using bfloat16 full finetuning which cuts memory usage by 50%.


model.safetensors:   0%|          | 0.00/536M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Full finetuning is enabled, so .get_peft_model has no effect


In [7]:
def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

# Get our previous dataset and format it:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/14234 [00:00<?, ? examples/s]

See result of the first row:

In [8]:
dataset[0]

{'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'What are the most significant changes in ransomware tactics reported for 2024, and how can an AWSbased organization detect and mitigate these new attack vectors?',
   'role': 'user'},
  {'content': 'The OpenText 2024 report highlights a shift from traditional encryptiononly ransomware to **dataexfiltrationonly** (extortionware) and **double/triple extortion** strategies. Attackers now prefer to steal sensitive data and threaten disclosure rather than encrypt it, which bypasses the effectiveness of robust backup programs. They also combine encryption, exfiltration, and direct pressure on customers or regulators, increasing reputational damage. In AWS environments, these tactics can be detected by monitoring **CloudTrail** for anomalous API calls that download large volumes of S3 objects or use `GetObject` and `CopyObject` excessively. GuardDuty can flag unusual data transfer patterns, and Amazon 

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [9]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 64,
        gradient_accumulation_steps = 8, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs=4,
        learning_rate = 1e-5,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc

        # Checkpoint configuration
        output_dir = "./checkpoints",           # Folder to save checkpoints
        save_strategy = "steps",                # Save based on steps
        save_steps = 40,                        # Save every 40 steps
        save_total_limit = 3,                   # Keep only last 3 checkpoints
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=36):   0%|          | 0/14234 [00:00<?, ? examples/s]

In [10]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3090. Max memory = 23.559 GB.
0.521 GB of memory reserved.


In [ ]:
%time trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 14,234 | Num Epochs = 4 | Total steps = 112
O^O/ \_/ \    Batch size per device = 64 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (64 x 8 x 1) = 512
 "-____-"     Trainable parameters = 268,098,176 of 268,098,176 (100.00% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,4.234600
2,4.256600
3,4.253900
4,4.133000
5,4.074100
6,3.848000
7,3.676800
8,3.572500
9,3.484700
10,3.418900


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("aws_finetuned_model_gemma_3_270m")
tokenizer.save_pretrained("aws_finetuned_model_gemma_3_270m")